# Multi Agent

## Common Code

### Load env

In [2]:
import dotenv

dotenv.load_dotenv()

True

### import ADK components

In [12]:
from google.adk.agents import Agent, SequentialAgent, ParallelAgent, LoopAgent
from google.adk.models.google_llm import Gemini
from google.adk.runners import InMemoryRunner
from google.adk.tools import google_search, AgentTool, FunctionTool
from google.genai import types

print("✅ ADK components imported successfully.")

✅ ADK components imported successfully.


In [4]:
retry_config=types.HttpRetryOptions(
    attempts=5,  # Maximum retry attempts
    exp_base=7,  # Delay multiplier
    initial_delay=1, # Initial delay before first retry (in seconds)
    http_status_codes=[429, 500, 503, 504] # Retry on these HTTP errors
)

# Coordinator Pattern (Orchestrator Pattern)
Root Agent 가 목표에 도달하기 위해 필요한 Agent 에게 작업을 위임하고 결과를 받아오는 방식을 반복하여 최종 결과를 만들어내는 방식 \
Tool 호출 여부 및 순서는 전적으로 Root Agent 에 맡겨진다.

## Workflow
![](../images/architecture/coordinator.png)

## Define Agents

### Agent 1: Research Agent
- Its job is to use the google_search tool and present findings.

In [32]:
research_agent = Agent(
    name="ResearchAgent",
    model=Gemini(
        model="gemini-2.5-flash-lite",
        retry_options=retry_config
    ),
    instruction="""You are a specialized research agent. Your only job is to use the
    google_search tool to find 2-3 pieces of relevant information on the given topic and present the findings with citations.""",
    tools=[google_search],
    output_key="research_findings",  # The result of this agent will be stored in the session state with this key.
)

print("✅ research_agent created.")

✅ research_agent created.


### Agent 2 : Summarizer Agent
- Its job is to summarize the text it receives.

In [33]:
summarizer_agent = Agent(
    name="SummarizerAgent",
    model=Gemini(
        model="gemini-2.5-flash-lite",
        retry_options=retry_config
    ),
    # The instruction is modified to request a bulleted list for a clear output format.
    instruction="""Read the provided research findings: {research_findings}
Create a concise summary as a bulleted list with 3-5 key points.""",
    output_key="final_summary",
)

print("✅ summarizer_agent created.")

✅ summarizer_agent created.


### Root Agent
- Orchestrates the workflow by calling the sub-agents as tools.


In [34]:
root_agent = Agent(
    name="RootAgent",
    model=Gemini(
        model="gemini-2.5-flash-lite",
        retry_options=retry_config
    ),
    # This instruction tells the root agent HOW to use its tools (which are the other agents).
    instruction="""You are a research coordinator. Your goal is to answer the user's query by orchestrating a workflow.
1. First, you MUST call the `ResearchAgent` tool to find relevant information on the topic provided by the user.
2. Next, after receiving the research findings, you MUST call the `SummarizerAgent` tool to create a concise summary.
3. Finally, present the final summary clearly to the user as your response.""",
    # We wrap the sub-agents in `AgentTool` to make them callable tools for the root agent.
    tools=[AgentTool(research_agent), AgentTool(summarizer_agent)],
)

print("✅ root_agent created.")

✅ root_agent created.


## Run

In [35]:
runner = InMemoryRunner(agent=root_agent)
response = await runner.run_debug(
    "양자 컴퓨팅의 최신 발전 동향은 무엇이며, 이는 인공지능에 어떤 의미를 지니는가?"
)


 ### Created new session: debug_session_id

User > 양자 컴퓨팅의 최신 발전 동향은 무엇이며, 이는 인공지능에 어떤 의미를 지니는가?
RootAgent > 양자 컴퓨팅은 큐비트라는 기본 단위를 사용하여 중첩과 얽힘 같은 양자역학 원리를 활용함으로써 기존 컴퓨터의 한계를 뛰어넘는 연산 능력을 제공하며, 2025년 상용화를 향한 가시적인 진전이 예상됩니다. IBM, Google 등 여러 기업과 연구기관에서 성능 개선을 위해 노력 중입니다.

양자 컴퓨팅은 AI의 대규모 데이터 학습 및 분석에 필요한 연산 능력의 한계를 극복하고, 더 빠르고 효율적인 머신러닝, 신경망, 대규모 언어 모델 개발을 가능하게 하여 신약 개발, 기후 과학 등 다양한 분야의 혁신을 이끌 것으로 기대됩니다.

현재는 오류가 있는 중간 규모의 양자 컴퓨터(NISQ)를 사용하고 있어 고전 AI를 완전히 능가하기는 어렵지만, 고전 컴퓨팅과 양자 컴퓨팅을 결합한 하이브리드 방식의 연구가 활발히 진행 중입니다. 2025년에는 실용적인 양자 응용이 가시화될 것으로 예상되며, 일부에서는 2025년 말까지 산업적으로 유용한 양자 컴퓨터가 등장할 가능성을 높게 보고 있습니다.


## Print data for debugging

In [48]:
for resp in response:
    print(resp.model_dump_json(indent=2))

{
  "model_version": "gemini-2.5-flash-lite",
  "content": {
    "parts": [
      {
        "media_resolution": null,
        "code_execution_result": null,
        "executable_code": null,
        "file_data": null,
        "function_call": {
          "id": "adk-38277395-1a1f-4b80-8a75-ed075e467ad2",
          "args": {
            "request": "양자 컴퓨팅의 최신 발전 동향과 인공지능에 미치는 영향"
          },
          "name": "ResearchAgent",
          "partial_args": null,
          "will_continue": null
        },
        "function_response": null,
        "inline_data": null,
        "text": null,
        "thought": null,
        "thought_signature": null,
        "video_metadata": null
      }
    ],
    "role": "model"
  },
  "grounding_metadata": null,
  "partial": null,
  "turn_complete": null,
  "finish_reason": "STOP",
  "error_code": null,
  "error_message": null,
  "interrupted": null,
  "custom_metadata": null,
  "usage_metadata": {
    "cache_tokens_details": null,
    "cached_content_token_

## 결과 분석
Root Agent -> Research Agent -> Root Agent -> Summarizer Agent -> Root Agent 순으로 호출되고 최종 응답 결과가 생성됨

### 문제점
Root Agent 에 도구 호출 여부 및 순서에 대한 판단을 전적으로 맡기기 때문에 특정 Tool 은 호출되지 않을 수도 있음 \
위의 예시에서는 Research Agent 가 알아서 정리하여 요약하면서 Root Agent 가 Summarizer Agent 가 필요없다고 판단하는 경우가 발생할 수 있음

### Solution
1. Tool 의 역할 제한. ex) 위의 예시에서 Research Agent 가 알아서 요약을 못하게 함으로써 Summarizer Agent 유도
2. Sequential Workflow 방식 채택

# Sequential Workflows Pattern (Pipeline Pattern, Assembly Line Pattern)
Agent 들을 일렬로 연결하여 강제적으로 순차적으로 실행하는 구조 \
이 방식은 앞 단계의 출력이 그대로 뒤 단계의 입력으로 들어감.

위의 Coordinator Pattern 과 다르게 Agent 실행의 순서와 실행여부가 코드에 의해 강제적으로 정해짐.

## Workflow
![](../images/architecture/sequential.png)

## Define Agents

### Agent 1: Outline Agent
**Concept**
Creates a blog outline for a given topic

In [49]:
outline_agent = Agent(
    name="OutlineAgent",
    model=Gemini(
        model="gemini-2.5-flash-lite",
        retry_options=retry_config
    ),
    instruction="""Create a blog outline for the given topic with:
    1. A catchy headline
    2. An introduction hook
    3. 3-5 main sections with 2-3 bullet points for each
    4. A concluding thought""",
    output_key="blog_outline",  # The result of this agent will be stored in the session state with this key.
)

print("✅ outline_agent created.")

✅ outline_agent created.


### Agent 2: Writer Agent
**Concept** Writes a blog post

In [50]:
writer_agent = Agent(
    name="WriterAgent",
    model=Gemini(
        model="gemini-2.5-flash-lite",
        retry_options=retry_config
    ),
    instruction="""Following this outline strictly: {blog_outline}
    Write a brief, 200 to 300-word blog post with an engaging and informative tone.""",
    output_key="blog_draft",  # The result of this agent will be stored with this key.
)

print("✅ writer_agent created.")

✅ writer_agent created.


### Agent 3: Editor Agent
**Concept** Polishes the blog draft

In [51]:
editor_agent = Agent(
    name="EditorAgent",
    model=Gemini(
        model="gemini-2.5-flash-lite",
        retry_options=retry_config
    ),
    instruction="""Edit this draft: {blog_draft}
    Your task is to polish the text by fixing any grammatical errors, improving the flow and sentence structure, and enhancing overall clarity.""",
    output_key="final_blog",  # This is the final output of the entire pipeline.
)

print("✅ editor_agent created.")

✅ editor_agent created.


In [52]:
root_agent = SequentialAgent(
    name="BlogPipeline",
    sub_agents=[outline_agent, writer_agent, editor_agent],
)

print("✅ Sequential Agent created.")

✅ Sequential Agent created.


## Run

In [53]:
runner = InMemoryRunner(agent=root_agent)
response = await runner.run_debug(
    "소프트웨어 개발자에게 다중 에이전트 시스템의 이점에 관한 블로그 글을 작성하세요."
)


 ### Created new session: debug_session_id

User > 소프트웨어 개발자에게 다중 에이전트 시스템의 이점에 관한 블로그 글을 작성하세요.
OutlineAgent > ## 다중 에이전트 시스템: 소프트웨어 개발의 게임 체인저 🚀

**소프트웨어 개발의 복잡성은 끝없이 증가하고 있습니다. 이러한 복잡성을 헤쳐나가고 더 효율적이며 강력한 솔루션을 구축하기 위한 새로운 방법론이 절실히 필요합니다. 여기서 다중 에이전트 시스템(Multi-Agent Systems, MAS)이 등장합니다. MAS는 개별적으로는 비교적 간단하지만, 상호 작용을 통해 복잡하고 지능적인 행동을 수행하는 자율 에이전트들의 집합입니다. 소프트웨어 개발자로서 MAS를 이해하고 활용하는 것은 미래 경쟁력 확보에 필수적입니다.**

### 1. 향상된 모듈성과 재사용성: 코드의 유연성을 높이다

*   **독립적인 개발 및 관리:** 각 에이전트는 독립적인 단위로 설계 및 개발될 수 있어, 팀원 간의 의존성을 줄이고 개발 속도를 높일 수 있습니다.
*   **모듈화된 기능:** 특정 기능을 담당하는 에이전트를 생성함으로써, 코드 베이스가 더욱 모듈화되고 이해하기 쉬워집니다. 이는 유지보수 및 업데이트를 용이하게 합니다.
*   **높은 재사용성:** 잘 정의된 에이전트는 다른 시스템이나 프로젝트에서 쉽게 재사용될 수 있어, 개발 시간과 비용을 절감하는 데 크게 기여합니다.

### 2. 분산 처리 및 확장성: 대규모 시스템을 거뜬히

*   **병렬 처리 능력:** 여러 에이전트가 동시에 작업을 수행할 수 있으므로, 복잡한 계산이나 데이터 처리 작업을 분산하여 처리하는 것이 가능합니다.
*   **수평적 확장 용이:** 시스템의 부하가 증가하면, 단순히 에이전트의 수를 늘리는 것만으로도 쉽게 확장할 수 있습니다. 이는 동적인 환경에 유연하게 대처할 수 있게 합니다.
*   **내결함성 증대:** 특정 에이전트에 장애가 발생하더라도, 다른 에이전트들이 작업을 계속 수

## Print data for debugging

In [54]:
for resp in response:
    print(resp.model_dump_json(indent=2))

{
  "model_version": "gemini-2.5-flash-lite",
  "content": {
    "parts": [
      {
        "media_resolution": null,
        "code_execution_result": null,
        "executable_code": null,
        "file_data": null,
        "function_call": null,
        "function_response": null,
        "inline_data": null,
        "text": "## 다중 에이전트 시스템: 소프트웨어 개발의 게임 체인저 🚀\n\n**소프트웨어 개발의 복잡성은 끝없이 증가하고 있습니다. 이러한 복잡성을 헤쳐나가고 더 효율적이며 강력한 솔루션을 구축하기 위한 새로운 방법론이 절실히 필요합니다. 여기서 다중 에이전트 시스템(Multi-Agent Systems, MAS)이 등장합니다. MAS는 개별적으로는 비교적 간단하지만, 상호 작용을 통해 복잡하고 지능적인 행동을 수행하는 자율 에이전트들의 집합입니다. 소프트웨어 개발자로서 MAS를 이해하고 활용하는 것은 미래 경쟁력 확보에 필수적입니다.**\n\n### 1. 향상된 모듈성과 재사용성: 코드의 유연성을 높이다\n\n*   **독립적인 개발 및 관리:** 각 에이전트는 독립적인 단위로 설계 및 개발될 수 있어, 팀원 간의 의존성을 줄이고 개발 속도를 높일 수 있습니다.\n*   **모듈화된 기능:** 특정 기능을 담당하는 에이전트를 생성함으로써, 코드 베이스가 더욱 모듈화되고 이해하기 쉬워집니다. 이는 유지보수 및 업데이트를 용이하게 합니다.\n*   **높은 재사용성:** 잘 정의된 에이전트는 다른 시스템이나 프로젝트에서 쉽게 재사용될 수 있어, 개발 시간과 비용을 절감하는 데 크게 기여합니다.\n\n### 2. 분산 처리 및 확장성: 대규모 시스템을 거뜬히\n\n*   **병렬 처리 능력:**

## 결과 분석
Outline Agent -> Writer Agent -> Editor Agent 순으로 호출되고 최종 응답 결과가 생성됨

### 문제점
속도가 느림

### Solution
연관된 작업이 아닌 경우 그냥 병렬로 처리되도록 하는 것이 좋음

# Parallel Workflows Pattern
Agent 들을 Sub-Agent 로 포함하여 병렬로 실행하는 ParallelAgent 와 \
각 Sub-Agent 들의 결과값을 종합하여 전체 결과값을 만드는 Aggregator Agent 를 Sequential Workflows Pattern 방식으로 연결된 구조

이 방식은 Agent 들끼리 간섭이 없음. \
Input, Output 이 공유가 불가능하며 완전히 독립된 채로 병렬로 실행되므로 서로 무관한 기능을 하는 Agent 일 때 의미가 있음

## Workflow
![](../images/architecture/parallel.png)

## Define Agents

### Agent 1: Tech Research Agent
**Concept**
Researches AI/ML news and trends

In [55]:
tech_researcher = Agent(
    name="TechResearcher",
    model=Gemini(
        model="gemini-2.5-flash-lite",
        retry_options=retry_config
    ),
    instruction="""Research the latest AI/ML trends. Include 3 key developments,
the main companies involved, and the potential impact. Keep the report very concise (100 words).""",
    tools=[google_search],
    output_key="tech_research",  # The result of this agent will be stored in the session state with this key.
)

print("✅ tech_researcher created.")

✅ tech_researcher created.


### Agent 2: Health Research Agent
**Concept**
Researches recent medical news and trends

In [56]:
health_researcher = Agent(
    name="HealthResearcher",
    model=Gemini(
        model="gemini-2.5-flash-lite",
        retry_options=retry_config
    ),
    instruction="""Research recent medical breakthroughs. Include 3 significant advances,
their practical applications, and estimated timelines. Keep the report concise (100 words).""",
    tools=[google_search],
    output_key="health_research",  # The result will be stored with this key.
)

print("✅ health_researcher created.")

✅ health_researcher created.


### Agent 3: Finance Research Agent
**Concept**
Researches finance and fintech news and trends

In [57]:
finance_researcher = Agent(
    name="FinanceResearcher",
    model=Gemini(
        model="gemini-2.5-flash-lite",
        retry_options=retry_config
    ),
    instruction="""Research current fintech trends. Include 3 key trends,
their market implications, and the future outlook. Keep the report concise (100 words).""",
    tools=[google_search],
    output_key="finance_research",  # The result will be stored with this key.
)

print("✅ finance_researcher created.")

✅ finance_researcher created.


### Agent 4: Aggregator Agent
**Concept**
Combines all research findings into a single summary

In [58]:
aggregator_agent = Agent(
    name="AggregatorAgent",
    model=Gemini(
        model="gemini-2.5-flash-lite",
        retry_options=retry_config
    ),
    # It uses placeholders to inject the outputs from the parallel agents, which are now in the session state.
    instruction="""Combine these three research findings into a single executive summary:

    **Technology Trends:**
    {tech_research}

    **Health Breakthroughs:**
    {health_research}

    **Finance Innovations:**
    {finance_research}

    Your summary should highlight common themes, surprising connections, and the most important key takeaways from all three reports. The final summary should be around 200 words.""",
    output_key="executive_summary",  # This will be the final output of the entire system.
)

print("✅ aggregator_agent created.")

✅ aggregator_agent created.


In [61]:
# The ParallelAgent runs all its sub-agents simultaneously.
parallel_research_team = ParallelAgent(
    name="ParallelResearchTeam",
    sub_agents=[tech_researcher, health_researcher, finance_researcher],
)

# This SequentialAgent defines the high-level workflow: run the parallel team first, then run the aggregator.
root_agent = SequentialAgent(
    name="ResearchSystem",
    sub_agents=[parallel_research_team, aggregator_agent],
)

print("✅ Parallel and Sequential Agents created.")

✅ Parallel and Sequential Agents created.


## Run

In [64]:
runner = InMemoryRunner(agent=root_agent)
response = await runner.run_debug(
    "기술, 헬스케어, 금융 분야의 일일 경영진 브리핑을 진행하세요."
)


 ### Created new session: debug_session_id

User > 기술, 헬스케어, 금융 분야의 일일 경영진 브리핑을 진행하세요.
HealthResearcher > ## 일일 경영진 브리핑: 기술, 헬스케어, 금융

**기술:**
*   **AI 기반 신약 개발 가속화:** 인공지능이 신약 후보 물질 발굴 및 임상 시험 설계 시간을 획기적으로 단축하며 제약 산업에 혁신을 가져오고 있습니다. 2025년까지 AI 기반 신약 출시가 가시화될 전망입니다.

**헬스케어:**
*   **개인 맞춤형 암 치료법 개발:** 유전체 분석 기술의 발전으로 환자 개개인의 종양 특성에 맞는 표적 치료제가 개발되어 치료 효과를 높이고 부작용을 최소화하고 있습니다. 2026년까지 더욱 정교해진 개인 맞춤형 암 치료법이 보편화될 것으로 예상됩니다.

**금융:**
*   **블록체인 기반 금융 거래 보안 강화:** 블록체인 기술이 금융 거래의 투명성과 보안성을 높이며 사기 및 해킹 위험을 줄이고 있습니다. 2027년까지 블록체인 기술이 주요 금융 시스템에 통합될 가능성이 높습니다.
TechResearcher > ## AI/ML 트렌드: 기술, 헬스케어, 금융 분야 경영진 브리핑

**주요 발전 동향:**

1.  **생성형 AI 및 멀티모달 AI:** 텍스트, 이미지, 비디오 등 다양한 형태의 콘텐츠를 생성하는 AI가 빠르게 발전하고 있습니다. 헬스케어에서는 신약 개발 및 진단 보조에, 금융에서는 맞춤형 상품 추천 및 사기 탐지에 활용될 잠재력이 큽니다.
2.  **AI 에이전트(Agentic AI):** 단순 지시 수행을 넘어 자율적으로 작업을 계획하고 실행하는 AI 에이전트가 주목받고 있습니다. 금융권에서는 자동화된 투자 분석 및 리스크 관리, 헬스케어에서는 환자 모니터링 및 예약 관리 등에 적용될 수 있습니다.
3.  **산업별 특화 AI 모델:** 범용 AI 모델에서 나아가 특정 산업의 데이터를 학습하고 전문화된 AI 모델을 개발하는 추세입니다. 금융

## Print data for debugging

In [65]:
for resp in response:
    print(resp.model_dump_json(indent=2))

{
  "model_version": "gemini-2.5-flash-lite",
  "content": {
    "parts": [
      {
        "media_resolution": null,
        "code_execution_result": null,
        "executable_code": null,
        "file_data": null,
        "function_call": null,
        "function_response": null,
        "inline_data": null,
        "text": "## 일일 경영진 브리핑: 기술, 헬스케어, 금융\n\n**기술:**\n*   **AI 기반 신약 개발 가속화:** 인공지능이 신약 후보 물질 발굴 및 임상 시험 설계 시간을 획기적으로 단축하며 제약 산업에 혁신을 가져오고 있습니다. 2025년까지 AI 기반 신약 출시가 가시화될 전망입니다.\n\n**헬스케어:**\n*   **개인 맞춤형 암 치료법 개발:** 유전체 분석 기술의 발전으로 환자 개개인의 종양 특성에 맞는 표적 치료제가 개발되어 치료 효과를 높이고 부작용을 최소화하고 있습니다. 2026년까지 더욱 정교해진 개인 맞춤형 암 치료법이 보편화될 것으로 예상됩니다.\n\n**금융:**\n*   **블록체인 기반 금융 거래 보안 강화:** 블록체인 기술이 금융 거래의 투명성과 보안성을 높이며 사기 및 해킹 위험을 줄이고 있습니다. 2027년까지 블록체인 기술이 주요 금융 시스템에 통합될 가능성이 높습니다.",
        "thought": null,
        "thought_signature": null,
        "video_metadata": null
      }
    ],
    "role": "model"
  },
  "grounding_metadata": {
    "google_maps_widget_context_token": null,
    "g

# Loop Workflows Pattern
작업 결과를 검증하는 Agent가 통과라고 판단할 때까지 주요 작업을 수행하는 Agent 가 반복적으로 수행하는 방법

## Workflow
![](../images/architecture/loop.png)

## Define Agents

### Agent 1: Initial Writer Agent
**Role**
첫 초안을 작성

In [5]:
initial_writer_agent = Agent(
    name="InitialWriterAgent",
    model=Gemini(
        model="gemini-2.5-flash-lite",
        retry_options=retry_config
    ),
    instruction="""Based on the user's prompt, write the first draft of a short story (around 100-150 words).
    Output only the story text, with no introduction or explanation.""",
    output_key="current_story",  # Stores the first draft in the state.
)

print("✅ initial_writer_agent created.")

✅ initial_writer_agent created.


### Agent 2: Critic Agent
**Role**
Story를 검토하고 피드백을 제공하는 비평가 역할. 만약 잘 쓰였다면 "APPROVED" 를 반환

In [7]:
critic_agent = Agent(
    name="CriticAgent",
    model=Gemini(
        model="gemini-2.5-flash-lite",
        retry_options=retry_config
    ),
    instruction="""You are a constructive story critic. Review the story provided below.
    Story: {current_story}

    Evaluate the story's plot, characters, and pacing.
    - If the story is well-written and complete, you MUST respond with the exact phrase: "APPROVED"
    - Otherwise, provide 2-3 specific, actionable suggestions for improvement.""",
    output_key="critique",  # Stores the feedback in the state.
)

print("✅ critic_agent created.")

✅ critic_agent created.


### Agent 3: Refiner Agent
**Role**
Critic Agent 로 부터 응답을 받고 수정하거나 종료하는 역할.

#### Loop Exit Function

In [8]:
def exit_loop():
    """Call this function ONLY when the critique is 'APPROVED', indicating the story is finished and no more changes are needed."""
    return {"status": "approved", "message": "Story approved. Exiting refinement loop."}

print("✅ exit_loop function created.")

✅ exit_loop function created.


#### define agent

In [11]:
refiner_agent = Agent(
    name="RefinerAgent",
    model=Gemini(
        model="gemini-2.5-flash-lite",
        retry_options=retry_config
    ),
    instruction="""You are a story refiner. You have a story draft and critique.

    Story Draft: {current_story}
    Critique: {critique}

    Your task is to analyze the critique.
    - IF the critique is EXACTLY "APPROVED", you MUST call the `exit_loop` function and nothing else.
    - OTHERWISE, rewrite the story draft to fully incorporate the feedback from the critique.""",
    output_key="current_story",  # It overwrites the story with the new, refined version.
    tools=[
        FunctionTool(exit_loop)
    ],  # The tool is now correctly initialized with the function reference.
)

print("✅ refiner_agent created.")

✅ refiner_agent created.


In [13]:
story_refinement_loop = LoopAgent(
    name="StoryRefinementLoop",
    sub_agents=[critic_agent, refiner_agent],
    max_iterations=2,  # Prevents infinite loops
)

# The root agent is a SequentialAgent that defines the overall workflow: Initial Write -> Refinement Loop.
root_agent = SequentialAgent(
    name="StoryPipeline",
    sub_agents=[initial_writer_agent, story_refinement_loop],
)

print("✅ Loop and Sequential Agents created.")

✅ Loop and Sequential Agents created.


In [15]:
runner = InMemoryRunner(agent=root_agent)
response = await runner.run_debug(
    "등대지기 한 명이 신비롭게 빛나는 지도를 발견하는 이야기를 담은 단편 소설을 써줘."
)


 ### Created new session: debug_session_id

User > 등대지기 한 명이 신비롭게 빛나는 지도를 발견하는 이야기를 담은 단편 소설을 써줘.
InitialWriterAgent > 밤마다 파도가 부서지는 소리가 유일한 동반자였던 늙은 등대지기 엘리아스는 낡은 궤짝에서 먼지 쌓인 양피지 한 장을 발견했다. 희미한 램프 불빛 아래 펼쳐진 지도는 낡고 바래 있었지만, 이상하게도 섬세한 푸른빛으로 희미하게 빛나고 있었다. 잉크로 그려진 선들은 마치 살아있는 것처럼 꿈틀거리는 듯했고, 별자리처럼 신비로운 문양들이 새겨져 있었다. 지도에는 알려지지 않은 섬들이 그려져 있었고, 그 중심에는 심연처럼 검은 글씨로 ‘별들의 눈물’이라 적힌 곳이 있었다. 엘리아스는 가슴이 뛰었다. 수십 년간 등대에서 보낸 삶이 그저 맹목적인 기다림이 아니었음을, 이 지도가 말해주고 있었다.
CriticAgent > The story has a strong atmospheric beginning and a compelling hook with the discovery of the mysterious map. However, it feels more like an introduction to a larger narrative than a complete story.

Here are a few suggestions for improvement:

1.  **Develop Elias's Internal Conflict and Motivation:** While the story mentions Elias's heart pounding and the feeling that his life wasn't a "blind wait," it would be stronger to delve deeper into *why* this map now stirs him. What was he waiting for? What are his dreams or regrets that th